In [ ]:
!pip uninstall fastai -y

In [ ]:
import sys,os

In [ ]:
sys.path.append('../input/fastaiv1')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import defaultdict
import os
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.image as immg
from sklearn.model_selection import StratifiedKFold,KFold

In [ ]:
!pip install object-detection-fastai

In [ ]:
from object_detection_fastai.helper.object_detection_helper import *
from object_detection_fastai.loss.RetinaNetFocalLoss import RetinaNetFocalLoss
from object_detection_fastai.models.RetinaNet import RetinaNet
from object_detection_fastai.callbacks.callbacks import BBLossMetrics, BBMetrics, PascalVOCMetric

In [ ]:
sns.set_style('darkgrid')

In [ ]:
df = pd.read_csv('../input/nfl-impact-detection/image_labels.csv')

In [ ]:
path = Path('../input/nfl-impact-detection')

In [ ]:
tr = df.image.value_counts()
tr = pd.DataFrame({'image':tr.index,'image_count':tr.values})
tr = tr.sample(frac=1.,random_state=2020).reset_index(drop=True)

In [ ]:
tr.head()

In [ ]:
df.sample(n=10)

In [ ]:
df_grp = df.groupby(['image'])

## Printing a sample

In [ ]:
name = '58146_002671_Endzone_frame304.jpg'
loc = '../input/nfl-impact-detection/images/'+name
temp = df_grp.get_group(name)
lb = temp.loc[:, (['label','left', 'width', 'top', 'height'])].values
img = immg.imread(loc)
fig,ax = plt.subplots(figsize=(20,12))
ax.imshow(img)
for s in lb:
    l,b = s[0],s[1:]
    rect = [b[0],b[2],b[1],b[3]]
    draw_rect(ax,rect,text=l,text_size=10,color='red')

In [ ]:
def get_lbl_img(train):
    helmet2bbox = {}
    grp = df.image.unique()
    tr_gr = df.groupby(['image'])
    from tqdm.notebook import tqdm
    for i in tqdm(range(len(grp))):
        name = str(grp[i])
        bbox = []
        lbls = []
        temp_b = []
        temp = tr_gr.get_group(grp[i])
        tt = temp.loc[:, (['label','left', 'width', 'top', 'height'])].values
        for j in range(len(temp)):
            lbls.append(tt[j][0])
            b = tt[j][1:]  
            t = [b[0],b[2],b[1],b[3]] # x,y, width, height
            # Currently our coordinates are x,w,l,h and we want x1,y1,x2,y2
            # To convert it, we need to add our width and height to the respective x and y.
            t[2],t[3] = t[0]+t[2],t[1]+t[3]  
            t1 = [t[1],t[0],t[3],t[2]]

            temp_b.append(t1)
        bbox.append(temp_b)
        bbox.append(lbls)
        helmet2bbox[name] = bbox
    return helmet2bbox

In [ ]:
helmet2bbox = get_lbl_img(df)

In [ ]:
chk = helmet2bbox['57802_001673_Endzone_frame0932.jpg']

In [ ]:
img = immg.imread(str(path/'images'/'57802_001673_Endzone_frame0932.jpg'))

## Sample Check 2

In [ ]:
fig,ax = plt.subplots(figsize=(16,10))
ax.imshow(img)
lbl,bbxs = chk[1],chk[0]
for l, b in zip(lbl,bbxs):
    rect = [b[0],b[1],b[3]-b[1],b[2]-b[0]]
    rect1 = [rect[1],rect[0],rect[3],rect[2]]
    draw_rect(ax,rect1,text=l,text_size=12,color='red')

In [ ]:
img.shape[0]/2,img.shape[1]/2

In [ ]:
get_y_func = lambda o: helmet2bbox[Path(o).name] 

## DataLoader

In [ ]:
data = (ObjectItemList.from_df(tr.sample(frac=0.5),path, folder = 'images' ,cols='image')
        #Where are the images? ->
        .split_by_rand_pct(0.2)                          
        #How to split in train/valid? -> randomly with the default 20% in valid
        .label_from_func(get_y_func)
        #How to find the labels? -> use get_y_func on the file name of the data
        .transform(size=512,resize_method=ResizeMethod.SQUISH)
        #.add_test(ts)
        #Data augmentation? -> Standard transforms; also transform the label images
        .databunch(bs=2, collate_fn=bb_pad_collate))

In [ ]:
data.show_batch( 1, figsize = (20,12))

In [ ]:
len(data.train_ds),len(data.valid_ds)

In [ ]:
data.classes,data.c

In [ ]:
size = 512

### Anchors

In [ ]:
anchors = create_anchors(sizes=[(32,32),(16,16),(8,8),(4,4)], ratios=[0.5, 1, 2], scales=[0.15, 0.25, 0.35, 0.45, 0.55, 0.75])

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))
ax.imshow(image2np(data.valid_ds[0][0].data))

for i, bbox in enumerate(anchors[:18]):
    bb = bbox.numpy()
    x = (bb[0] + 1) * size / 2 
    y = (bb[1] + 1) * size / 2 
    w = bb[2] * size / 2
    h = bb[3] * size / 2
    
    rect = [x,y,w,h]
    draw_rect(ax,rect)

In [ ]:
len(anchors)

## Encoder and Model

In [ ]:
n_classes = data.train_ds.c

crit = RetinaNetFocalLoss(anchors)

encoder = create_body(models.resnet18, True, -2)

model = RetinaNet(encoder, n_classes=data.train_ds.c, n_anchors=18, sizes=[32,16,8,4], chs=32, final_bias = -4., n_conv = 2)

## Learner

In [ ]:
voc = PascalVOCMetric(anchors, size, [i for i in data.train_ds.y.classes[1:]])
learn = Learner(data,
                model, 
                loss_func=crit,
                callback_fns=[BBMetrics],
                metrics=[voc],
                model_dir = '/kaggle/working/')

In [ ]:
learn.split([model.encoder[6], model.c5top5]);
learn.freeze_to(-2)
#learn = learn.to_fp16()

In [ ]:
#learn.unfreeze()
learn.fit_one_cycle(3, 1e-3 ,callbacks = [SaveModelCallback(learn, every ='improvement', monitor ='AP-Helmet', name ='best_model',mode='max')])

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.load('best_model');

## Results

In [ ]:
show_results_side_by_side(learn, anchors, detect_thresh=0.3, nms_thresh=0.1, image_count=10)